In [24]:
#FIRST of all, install SSMS (SQL Server MicroSoft)
#Second of all, open the SQLQuery1.sql file in SqlServer
#Thrid of all, highlight ONLY the first command (create database Brokerage) then click Execute
#Fourth of all, highlight the rest of the file lines then click Execute.
#by now you should have the database and schema set and done

#now you will need to populate it with data

#Fifth of all, install pyodbc (nice library btw)

#you will then want to connect your database to the ODBC:
#   1) open SQL Server
#   2) right click on the very first element in object explorer (which is your device obvs)
#   3) choose connect
#   4) copy the server name
#   5) paste it in the server string below


#Sixth of all, just run everything below and enjoy :)

#OPTIONAL : you don't need to know how to generate the report file, it is so ez though
#   just open SSMS
#   Select the database "Brokerage"
#   Right click and select Reports then Standard Reports
#   Select any of the reports than seems interesting

In [1]:
import pandas as pd
import pyodbc
import numpy as np
import random
from tqdm import tqdm


In [10]:
#DELETE THIS AFTER EXECUTION

Address_df = pd.read_csv('Street_Address_Listing.csv')
while (len(Address_df) < 1000000):
    Address_df = Address_df.append(Address_df)

Address_df.to_csv('Address.csv')

In [12]:
#DELETE THIS AFTER EXECUTION

names_df = pd.read_csv('NationalNames.csv')
while (len(names_df) < 1000000):
    names_df = names_df.append(names_df)

names_df.to_csv('Names.csv')

Department_df = pd.read_csv('2019_free_title_data.csv')
while(len(Department_df)<1000000):
    Department_df = Department_df.append(Department_df)
Department_df.to_csv('Jobs.csv')

In [2]:
#Helper Functions



alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

#to return a random Department Name
def GenerateRandName(digits):
    Random_dept_name = ''
    max_len = random.choice(range(9,40))
    while len(Random_dept_name) < max_len:
        if(digits):
            Random_dept_name = Random_dept_name+(random.choice(alphabet+'123456789'))
        else:
            Random_dept_name = Random_dept_name+(random.choice(alphabet))
    return Random_dept_name

def GenerateRandomDate():
    randDay = random.choice(range(1,29))
    randMonth = random.choice(range(1,12))
    randYear = random.choice(range(2000,2023))
    return pd.to_datetime(str(randDay) + "/"+str(randMonth)+"/"+str(randYear), dayfirst=True)


In [4]:
for driver in pyodbc.drivers():
    print(driver)

SQL Server
SQL Server Native Client 11.0
SQL Server Native Client RDA 11.0
ODBC Driver 17 for SQL Server
Microsoft Access Driver (*.mdb, *.accdb)
Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)
Microsoft Access Text Driver (*.txt, *.csv)


In [3]:
df = pd.read_csv('Employee.csv')
#Columns in the CSV : first_name Minit last_name Address gender BDate Salary
#attributes in our database: Fname Minit Lname Bdate Address_ Sex Salary
#print(df['first_name'])

#for i in range(1000):
#    df.loc[i,'Minit'] = random.choice(alphabet)
#df.to_csv('MOCK_DATA.csv')

#print(df.iloc[:,:].values.tolist())


In [ ]:
#Iam not sure whether this cell is outdated or not



names_df = pd.read_csv('Names.csv')
Address_df = pd.read_csv('Address.csv')
df = pd.DataFrame()


for i in tqdm(range(30000), desc="making Employee.csv: "):
    df.loc[i, 'first_name'] = names_df.loc[i%1000000, 'Name']
    df.loc[i, 'Minit'] = alphabet[i%27]
    df.loc[i, 'last_name'] = names_df.loc[(i+1)%1000000, 'Name']
    df.loc[i, 'Address'] = Address_df.loc[i%1000000, 'FULL ADDRESS']

    df.loc[i, 'BDate'] = pd.to_datetime(str((i%28) + 1) + "/"+str((i%12) + 1)+"/"+str(2000 + i%40), dayfirst=True)
    df.loc[i, 'Sex'] = i%3
    df.loc[i, 'Salary'] = i*3000


df.to_csv('Employee.csv')


In [4]:
#QUERIES TO FILL "Employee"


server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()


insert_to_employee = 'INSERT INTO Employee (Fname, Minit, Lname, Address_, Bdate, Sex, Salary) VALUES (?,?,?, ?, ?, ?, ?)'
select_all_from_employee = "select * from Employee"

for i in tqdm(range(1000), desc="inserting data into Table Employee: "):
    #lst = [df['first_name'], df['Minit'], df['last_name'], pd.to_datetime(df['BDate']), df['Address'], df['gender'].astype(int), df['Salary'].astype(int)]

    #batch size is 3000
    lst = df.iloc[(i)*3000: (i+1)*3000,1:].values.tolist()
    cursor.executemany(insert_to_employee, lst)

connection.commit()

get_Employees_count = "select count (*) from Employee"

cursor = connection.cursor()
count = cursor.execute(get_Employees_count)
print(list(count))
#for row in cursor:
#    print(row)

inserting data into Table Employee: 100%|██████████| 1000/1000 [06:40<00:00,  2.50it/s]


[(3000000, )]


In [5]:
#Testing by inserting only one row into employees


#lst = [df['first_name'][0], df['Minit'][0], df['last_name'][0], pd.to_datetime(df['BDate'][0]), df['Address'][0], int(df['gender'][0]), int(df['Salary'][0])]


lst = [df.loc[0,'first_name'], df.loc[0,'Minit'], df.loc[0,'last_name'], df.loc[0,'BDate'], df.loc[0,'Address'], df.loc[0,'Sex'], df.loc[0,'Salary']]

server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()
cursor.execute("INSERT INTO Employee (Fname, Minit, Lname, Bdate, Address_, Sex, Salary) VALUES(?,?,?, ?, ?, ?, ?)", lst)

In [ ]:
'''
create table Contracts
(
Name_Company nvarchar(50),
Date_created DATE,
ProjectID int,
Product nvarchar(50),
Price int,
Date_of_Contract DATE,
Emp_Name nvarchar(50),
Emp_SSN int,
Contractor_Name nvarchar(50),
Cont_SSN int,
Delivery_Location nvarchar(50),
CONSTRAINT Contracts_KEY_COMPOSITE PRIMARY KEY (Product, Emp_SSN, Cont_SSN)
);
'''
names_df = pd.read_csv('Names.csv')
location_pd = pd.read_csv('Address.csv')
print('Done reading files')

Contract_df = pd.DataFrame()
Contract_df[0, 'Name_Company'] = "Alaraby"
Contract_df['Date_created'] = pd.to_datetime('11/11/2000', dayfirst= True)
Contract_df['ProjectID'] = 1
Contract_df['Product'] = "Cement"
Contract_df['Price'] = 2000
Contract_df['Date_of_Contract'] = pd.to_datetime('11/11/2000', dayfirst=True)
Contract_df['Emp_Name'] = "AYman"
Contract_df['Emp_SSN'] = 1
Contract_df['Contractor_Name'] = "Mohamed"
Contract_df['Cont_SSN'] = 1
Contract_df['Delivery_Location'] = "Helwan"




   
temp_df = Contract_df
for i in tqdm(range(1, 1000000),desc="making Contracts.csv: "):
    temp_df.loc[i, 'Name_Company'] = GenerateRandName(False)
    temp_df.loc[i, 'ProjectID'] = i+1
    temp_df.loc[i, 'Product'] = 'Prod_'+(GenerateRandName(True))

    temp_df.loc[i, 'Price'] = random.choice(range(1,2000000))
    temp_df.loc[i, 'Product'] = random.choice(range(1,200000))
    temp_df.loc[i, 'Emp_Name'] = names_df.loc[i, 'Name']
    temp_df.loc[i, 'Emp_SSN'] = i+1
    temp_df.loc[i, 'Contractor_Name'] = names_df.loc[i-1, 'Name']
    temp_df.loc[i, 'Cont_SSN'] = 4000000+i
    temp_df.loc[i, 'Delivery_Location'] = location_pd.loc[i, 'FULL ADDRESS']
    
    temp_df.loc[i, 'Date_created'] = GenerateRandomDate()
    temp_df['Date_of_Contract'] = GenerateRandomDate()
    if(i % 10000 == 0 ):
        Contract_df = pd.concat((Contract_df,temp_df), axis=0)
        temp_df = pd.DataFrame()
        Contract_df.to_csv('Contracts.csv')

#Contract_df.to_csv('Contracts.csv')

In [33]:
for i in tqdm(range(1000000), desc="modifying contracts"):
    Contract_df.loc[i,'Emp_SSN'] = i+1

Contract_df = Contract_df.iloc[:,1:]
Contract_df.to_csv('Contracts.csv')
Contract_df = pd.read_csv('Contracts.csv')
print(Contract_df.head())

modifying contracts: 100%|██████████| 1000000/1000000 [10:40<00:00, 1562.32it/s]


   Unnamed: 0  (0, 'Name_Company') Date_created  ProjectID  Product  \
0           0                  NaN   2013-05-21        2.0    63999   
1           1                  NaN   2011-10-03        3.0    37891   
2           2                  NaN   2010-02-24        4.0    88587   
3           3                  NaN   2015-08-24        5.0   121650   
4           4                  NaN   2019-02-16        6.0   162283   

       Price Date_of_Contract   Emp_Name  Emp_SSN Contractor_Name   Cont_SSN  \
0  1063907.0       2000-10-21       Anna      1.0            Mary  4000001.0   
1  1381986.0       2000-10-21       Emma      2.0            Anna  4000002.0   
2   339553.0       2000-10-21  Elizabeth      3.0            Emma  4000003.0   
3  1849411.0       2000-10-21     Minnie      4.0       Elizabeth  4000004.0   
4   755588.0       2000-10-21   Margaret      5.0          Minnie  4000005.0   

                      Delivery_Location                           Name_Company  
0          

In [38]:
connection.commit()
cursor.execute('delete from Contracts')
connection.commit()

In [39]:
server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()

Contract_df = pd.read_csv('Contracts.csv')
print(Contract_df.head())

insert_to_Contracts= "INSERT INTO Contracts (Date_created, ProjectID, Product, Price, Date_of_Contract, Emp_Name, Emp_SSN, Contractor_Name, Cont_SSN, Delivery_Location,Name_Company) VALUES(?,?,?, ?,?,?,?,?,?,?,?)"
select_all_from_Contracts = "select * from Contracts"
get_Contracts_count = "select count (*) from Contracts"


for i in tqdm(range(0,900), desc="inserting data into table Contracts"):
    lst = Contract_df.iloc[i*1000:(i+1)*1000,2:].values.tolist()
    cursor.executemany(insert_to_Contracts, lst)


connection.commit()

count = cursor.execute(get_Contracts_count)
print(list(count))

   Unnamed: 0  (0, 'Name_Company') Date_created  ProjectID  Product  \
0           0                  NaN   2013-05-21        2.0    63999   
1           1                  NaN   2011-10-03        3.0    37891   
2           2                  NaN   2010-02-24        4.0    88587   
3           3                  NaN   2015-08-24        5.0   121650   
4           4                  NaN   2019-02-16        6.0   162283   

       Price Date_of_Contract   Emp_Name  Emp_SSN Contractor_Name   Cont_SSN  \
0  1063907.0       2000-10-21       Anna      1.0            Mary  4000001.0   
1  1381986.0       2000-10-21       Emma      2.0            Anna  4000002.0   
2   339553.0       2000-10-21  Elizabeth      3.0            Emma  4000003.0   
3  1849411.0       2000-10-21     Minnie      4.0       Elizabeth  4000004.0   
4   755588.0       2000-10-21   Margaret      5.0          Minnie  4000005.0   

                      Delivery_Location                           Name_Company  
0          

inserting data into table Contracts: 100%|██████████| 900/900 [02:32<00:00,  5.92it/s]


[(900000, )]


In [4]:
''' Department Attributes
Dname nvarchar(50),
Dnumber int identity (1,1) Primary Key NOT NULL,
Mgr_SSN int foreign key references Employee(SSN),
Mgr_Start_Date DATE
'''
Dept_df = pd.DataFrame()
Dept_df['Dname'] = GenerateRandName(True)
#Dept_df['Dnumber'] = 0
Dept_df['Mgr_SSN'] = random.choice(range(1,30))
Dept_df['Mgr_Start_Date'] = pd.to_datetime('11/11/2000', dayfirst=True)

temp_df = Dept_df
for i in tqdm(range(1,10000), desc="making Department.csv: "):
    temp_df.loc[i, 'Dname'] = GenerateRandName(True)
#    temp_df.loc[i, 'Dnumber'] = i
    temp_df.loc[i, 'Mgr_SSN'] = random.choice(range(1,2000000))
    temp_df.loc[i, 'Mgr_Start_Date'] = GenerateRandomDate()
    


Dept_df = pd.concat((Dept_df, temp_df), axis=0)

Dept_df = pd.read_csv('Department.csv')
while(len(Dept_df) < 1200000):
    Dept_df = pd.concat((Dept_df, Dept_df),axis=0)
Dept_df.to_csv('Department.csv')

Dept_df['Dname'].replace('', np.nan, inplace=True)
Dept_df['Mgr_SSN'].replace('', np.nan, inplace=True)
Dept_df['Mgr_Start_Date'].replace('', np.nan, inplace=True)

Dept_df.dropna(subset=['Dname'], inplace=True)
Dept_df.dropna(subset=['Mgr_SSN'], inplace=True)
Dept_df.dropna(subset=['Mgr_Start_Date'], inplace=True)



making Department.csv: 100%|██████████| 9999/9999 [00:10<00:00, 966.27it/s] 


In [14]:
server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()
Dept_df = pd.read_csv('Department.csv')

insert_to_Department= "INSERT INTO Department (Dname, Mgr_SSN, Mgr_Start_Date) VALUES(?,?, ?)"
select_all_from_Department = "select * from Department"
get_Department_count = "select count (*) from Department"

for i in tqdm(range (1000), desc="inserting data into Department: "):
    lst = Dept_df.iloc[i*1000:(i+1)*1000,1:].values.tolist()
    cursor.executemany(insert_to_Department, lst)
    

connection.commit()

count = cursor.execute(get_Department_count)
print(list(count))

inserting data into Department: 100%|██████████| 1000/1000 [02:17<00:00,  7.25it/s]

[(1109999, )]


In [42]:
connection.commit()
cursor.execute('delete from Dept_Locations')
connection.commit()

In [3]:
''' DEPT LOCATION

create table Dept_Locations
(
Dnumber INT foreign key references Department(Dnumber),
Dlocation nvarchar(50) NOT NULL,
CONSTRAINT Department_KEY_COMPOSITE PRIMARY KEY (Dnumber, Dlocation)
);
'''

DeptLoc_df = pd.DataFrame()
Address_df = pd.read_csv('Address.csv')

temp_df = DeptLoc_df
for i in tqdm(range(900000), desc="Making Dept_Locations.csv: "):
    temp_df.loc[i,'Dnumber'] = i+1
    temp_df.loc[i,'Dlocation'] = Address_df.loc[random.choice(range(1,400000)), 'FULL ADDRESS']
    if(i%20000 == 0):
        DeptLoc_df = pd.concat((DeptLoc_df, temp_df), axis=0)
        temp_df = pd.DataFrame()
        DeptLoc_df.to_csv('Dept_Locations.csv')



DeptLoc_df = pd.concat((DeptLoc_df, temp_df), axis=0)
DeptLoc_df.to_csv('Dept_Locations.csv')

Making Dept_Locations.csv: 100%|██████████| 900000/900000 [17:09<00:00, 873.97it/s]  


In [59]:
connection.commit()
cursor.execute('Delete from Dept_Locations')
connection.commit()

In [60]:

server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()

DeptLoc_df = pd.read_csv('Dept_Locations.csv')
print(DeptLoc_df.head())

insert_to_DeptLocation = "INSERT INTO Dept_Locations (Dlocation) VALUES(?)"
select_all_from_DeptLocations = "select * from Dept_Locations"
get_Dept_location_count = "select count (*) from Dept_Locations"



for i in tqdm(range(950),desc="inserting into Dept_Locations: "):
    lst = DeptLoc_df.iloc[i*1000:(i+1)*1000,2].values.tolist()
    for s in lst:
        try:
            cursor.execute(insert_to_DeptLocation, s)
        except:
            pass
   


connection.commit()

count = cursor.execute(get_Dept_location_count)
print(list(count))

   Unnamed: 0  Dnumber                  Dlocation
0           1      1.0          10082 E POMONA DR
1           2      2.0          16933 TENEIYA AVE
2           3      3.0          13630 LANDMARK DR
3           4      4.0          11733 SUN BELT CT
4           5      5.0  17100-17400 BLACKWATER RD


inserting into Dept_Locations: 100%|██████████| 950/950 [03:39<00:00,  4.32it/s]


[(0, )]


In [ ]:

''' Intern TABLE ATTRIBUTES
(
SSN int identity (3000000,1) primary key Not NULL,
Fname nvarchar(50),
Minit varchar,
Lname nvarchar(50),
Bdate DATE,
Address_ nvarchar(50),
Sex tinyint check(Sex between 0 and 2),
Salary int,
Major nvarchar(50),
PNO INT foreign key references Project(Pnumber),
Super_SSN int foreign key references Employee(SSN)
);
'''

names_df = pd.read_csv('Names.csv')
Address_df = pd.read_csv('Address.csv')
jobs_df = pd.read_csv('Jobs.csv')

#new empty dataframe
df_intern = pd.Dataframe()
#df_intern['SSN']=         auto incremented
df_intern['Fname']= ['Ayman', 'Mohamed']
df_intern['Minit']= ['M', 'R']
df_intern['Lname']= ['Reda', 'Abdo']
df_intern['Bdate']= [pd.to_datetime("20/12/2004", dayfirst=True), pd.to_datetime("11/11/2000", dayfirst=True)]
df_intern['Address_']= [Address_df.loc[1, 'FULL ADDRESS'], Address_df.loc[3, 'FULL ADDRESS']]
df_intern['Sex']= [1,1]
df_intern['Salary']= [2000,3000]
df_intern['Major']= ["Computer", "Engineering"]
df_intern['PNO']= [3, 5]                                                                #MUST BE PRESENT IN THE PNO
df_intern['Super_SSN']= [1, 2]


temp_df = df_intern
for i in tqdm(range(3001000),desc="making Intern.csv: "):
    temp_df.loc[i, 'Fname'] = names_df.loc[i%1000000, 'Name']
    temp_df.loc[i, 'Minit'] = random.choice(alphabet)
    temp_df.loc[i, 'Lname'] = names_df.loc[i%1000000, 'Name']
    temp_df.loc[i, 'Address_'] = Address_df.loc[i%1000000, 'FULL ADDRESS']
    
    randSex = random.choice(range(0,3))
    randSalary = random.choice(range(1, 100000))
    
    temp_df.loc[i, 'Bdate'] = GenerateRandomDate()
    temp_df.loc[i, 'Sex'] = randSex
    temp_df.loc[i, 'Salary'] = randSalary
    temp_df.loc[i, 'Major'] = jobs_df.loc[i%5000, 'top related titles 19']
    temp_df.loc[i, 'PNO'] = i%60000                                                               #Specify we have 60000 project
    temp_df.loc[i, 'Super_SSN'] = random.choice(range(1,500000))                                  #the first 500000 employees are intern supervisors
    if(i%17000 == 0):
        df_intern = pd.concat((df_intern, temp_df), axis=0)
        temp_df = pd.DataFrame()
        df_intern.to_csv('Intern.csv')

df_intern = pd.concat((df_intern, temp_df), axis=0)
df_intern.to_csv('Intern.csv')

In [62]:
connection.commit()
cursor.execute('delete from Intern')
connection.commit()


In [63]:

server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()

df_intern = pd.read_csv('Intern.csv')
print(df_intern.head())

insert_to_Intern = "INSERT INTO Intern (Fname, Minit, Lname, Bdate, Address_, Sex, Salary, Major, Super_SSN) VALUES(?,?, ?, ?, ?, ?, ?, ?, ?)"
select_all_from_Intern = "select * from Intern"
get_Intern_count = "select count (*) from Intern"
get_Employees_count = "select count (*) from Employee"

for i in tqdm(range(1000),desc="inserting into Intern: "):
    lst = df_intern.iloc[i*3000:(i+1)*3000,1:].values.tolist()
    cursor.executemany(insert_to_Intern, lst)
    

connection.commit()

count = cursor.execute(get_Employees_count)
print(list(count))

   Unnamed: 0      Fname Minit      Lname       Bdate  \
0           0       Anna     C       Anna  2000-01-25   
1           1       Emma     P       Emma  2001-02-24   
2           2  Elizabeth     g  Elizabeth  2000-04-01   
3           3     Minnie     y     Minnie  2000-03-27   
4           4   Margaret     U   Margaret  2013-05-21   

                               Address_  Sex   Salary                  Major  \
0               9007 HIGHLAND RD, STE 9  1.0  57209.0          sales manager   
1   5830 S SHERWOOD FOREST BLVD, STE A6  2.0  83730.0              principal   
2  4520 S SHERWOOD FOREST BLVD, STE 103  2.0  82711.0                  owner   
3                 8334 O'HARA CT, STE D  2.0  53267.0   chief executive offi   
4               4250 BLOUNT RD, UNIT 14  0.0  38652.0           math teacher   

   Super_SSN  
0   220600.0  
1   390500.0  
2    79200.0  
3     3700.0  
4   231100.0  


inserting into Intern:  41%|████      | 410/1000 [03:37<05:13,  1.88it/s]


ProgrammingError: The second parameter to executemany must not be empty.

In [13]:
'''

create table Works_On
(
ESSN INT foreign key references Employee(SSN),
PNO INT foreign key references Project(Pnumber),
Hours INT,
CONSTRAINT WorksOn_KEY_COMPOSITE PRIMARY KEY (ESSN, PNO)
);

'''

#new empty dataframe
df_workson = pd.DataFrame()

df_workson[0,'ESSN']= 1
df_workson[0,'PNO']= 1
df_workson[0,'Hours']= 19

PNO_counter = 1
tempdf = df_workson
for i in tqdm(range(2900000), desc="making Works_On.csv: "):
    tempdf.loc[i, 'ESSN'] = i+2
    tempdf.loc[i, 'PNO'] = PNO_counter+1
    tempdf.loc[i, 'Hours'] = random.randint(15, 50)
    if(i%17000 == 0):
        PNO_counter = PNO_counter + 1
        df_workson = pd.concat((df_workson, tempdf),axis=0)
        tempdf = pd.DataFrame()
        df_workson.to_csv('Works_On.csv')

df_workson = pd.concat((df_workson, tempdf),axis=0)
df_workson.to_csv('Works_On.csv')

making Works_On.csv: 100%|██████████| 2900000/2900000 [53:17<00:00, 907.09it/s]  


In [21]:
connection.commit()
cursor.execute('delete from Works_On')
connection.commit()

df_workson = df_workson.iloc[1:,4:]
df_workson.to_csv('Works_On.csv')
df_workson = pd.read_csv('Works_On.csv')
print(df_workson.head())

   Unnamed: 0  ESSN  PNO  Hours
0           1   1.0  1.0   46.0
1           2   2.0  2.0   44.0
2           3   3.0  2.0   45.0
3           4   4.0  2.0   24.0
4           5   5.0  2.0   20.0


In [23]:
server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()
df_workson = pd.read_csv('Works_On.csv')
print(df_workson.head())
insert_to_Works_On = "INSERT INTO Works_On (ESSN, PNO, Hours) VALUES(?,?,?)"
select_all_from_Works_On = "select * from Works_On"
get_Works_On_count = "select count (*) from Works_On"


for i in tqdm(range(965), desc="inserting into Works_On: "):
    lst = df_workson.iloc[i*3000:(i+1)*3000,1:].values.tolist()
    cursor.executemany(insert_to_Works_On, lst)

connection.commit()

count = cursor.execute(get_Works_On_count)
print(list(count))

   Unnamed: 0  ESSN  PNO  Hours
0           1   1.0  1.0   46.0
1           2   2.0  2.0   44.0
2           3   3.0  2.0   45.0
3           4   4.0  2.0   24.0
4           5   5.0  2.0   20.0


inserting into Works_On: 100%|██████████| 965/965 [04:34<00:00,  3.52it/s]


[(0, )]


In [ ]:
'''

create table Dependent_
(
ESSN INT foreign key references Employee(SSN),
Dependent_Name nvarchar(50) NOT NULL,
Sex tinyint check(Sex between 0 and 2),
Bdate DATE,
Relationship nvarchar(50),
CONSTRAINT Dependent_KEY_COMPOSITE PRIMARY KEY (ESSN, Dependent_Name)
);
'''


Relationship_df = pd.read_csv('Relationship.csv')

#new empty dataframe
df_Dependent = pd.DataFrame()

df_Dependent[0,'ESSN']= 1
df_Dependent[0,'Dependent_Name']= "Akram"
df_Dependent[0,'Sex']= 1
df_Dependent[0,'Bdate']= pd.to_datetime('11/11/2000', dayfirst=True)
df_Dependent[0,'Relationship']= "Father"




temp_df = df_Dependent
for i in tqdm(range(2900000), desc="making Dependent.csv : "):
    temp_df.loc[i, 'ESSN'] = i+1
    temp_df.loc[i, 'Dependent_Name'] = GenerateRandName(False)
    temp_df.loc[i, 'Sex'] = random.randint(0, 2)
    temp_df.loc[i, 'Bdate'] = GenerateRandomDate()
    temp_df.loc[i, 'Relationship'] = Relationship_df.loc[random.randint(0,970),'Relationship']
    if(i%17000 == 0):
        df_Dependent = pd.concat((df_Dependent, temp_df), axis=0)
        temp_df = pd.DataFrame()
        df_Dependent.to_csv('Dependent.csv')


df_Dependent = pd.concat((df_Dependent, temp_df), axis=0)
df_Dependent.to_csv('Dependent.csv')

In [6]:
connection.commit()
cursor.execute("delete from Dependent_")
connection.commit()

In [7]:
server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()

df_Dependent= pd.read_csv('Dependent.csv')
print(df_Dependent.head())

insert_to_Dependent_ = "INSERT INTO Dependent_ (ESSN, Dependent_Name, Sex, Bdate, Relationship) VALUES(?,?,?,?,?)"
select_all_from_Dependent_ = "select * from Dependent_"
get_Dependent__count = "select count (*) from Dependent_"


for i in tqdm(range(960), desc="inserting into Dependent_ : "):
    lst = df_Dependent.iloc[i*3000:(i+1)*3000,2:].values.tolist()
    cursor.executemany(insert_to_Dependent_, lst)

connection.commit()

count = cursor.execute(get_Dependent__count)
print(list(count))

   Unnamed: 0.1  Unnamed: 0  ESSN                      Dependent_Name  Sex  \
0             0           0   1.0        paZKVTQLByxdXhOCpyxXxLUpiqqj  1.0   
1             1           1   2.0       FMQDdqvlxinFUqMVSuGIbqbkLorMc  1.0   
2             2           2   3.0            CjuVAbakpmWAYMFsHoCqobKc  2.0   
3             3           3   4.0  MmGysbOQtdhSRszpXLMzszspphVNUHeuzT  0.0   
4             4           4   5.0                VVXLuargiWnLGulRAkVD  2.0   

        Bdate Relationship  
0  2019-09-06       Lotlux  
1  2001-07-06  Mat Lam Tam  
2  2005-08-17     Wrapsafe  
3  2001-06-24     Treeflex  
4  2017-09-12       Vagram  


inserting into Dependent_ : 100%|██████████| 960/960 [04:02<00:00,  3.95it/s]


[(2880000, )]


In [19]:

server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()

project_df = pd.read_csv('Project.csv')
print(project_df.head())

insert_to_Project = 'INSERT INTO Project (Pname, Plocation, DNO) VALUES (?,?, ?)'
select_all_from_Project = "select * from Project"

for i in tqdm(range(980), desc="inserting data into Table Project: "):
    lst = project_df.iloc[(i)*1000: (i+1)*1000,1:].values.tolist()
    cursor.executemany(insert_to_Project, lst)

connection.commit()

get_Project_count = "select count (*) from Project"

cursor = connection.cursor()
count = cursor.execute(get_Project_count)
print(list(count))

   Unnamed: 0                                           Pname  \
0           0                              advanced databases   
1           1                                              9F   
2           2                           LEZ3SwJIykqigwhJ1bPw7   
3           3             Vyp3AQZxOF4zQNki7hAU5m5B7X4gY1o3cy6   
4           4  ffI4FhNEyXiK9ri5QToVRIBkxg5pUCgOV1m7BB868r2nm8   

                              Plocation  DNO  
0           7353 HIGHLAND RD, STE B 282  1.0  
1               9007 HIGHLAND RD, STE 9  2.0  
2   5830 S SHERWOOD FOREST BLVD, STE A6  3.0  
3  4520 S SHERWOOD FOREST BLVD, STE 103  4.0  
4                 8334 O'HARA CT, STE D  5.0  


inserting data into Table Employee: 100%|██████████| 980/980 [02:39<00:00,  6.16it/s]


[(980000, )]


In [64]:
data = cursor.execute('select count(*) from Employee')
print("size of Employee : " + str(list(data)))

data = cursor.execute('select count(*) from Department')
print("size of Department : " + str(list(data)))


data = cursor.execute('select count(*) from Project')
print("size of Project : " + str(list(data)))

data = cursor.execute('select count(*) from Intern')
print("size of Intern : " + str(list(data)))

data = cursor.execute('select count(*) from Contracts')
print("size of Contracts : " + str(list(data)))

data = cursor.execute('select count(*) from Works_On')
print("size of Works_On : " + str(list(data)))

data = cursor.execute('select count(*) from Dept_Locations')
print("size of Dept_Locations : " + str(list(data)))


size of Employee : [(3010000, )]
size of Department : [(1109999, )]
size of Project : [(980000, )]
size of Intern : [(1228736, )]
size of Contracts : [(900000, )]
size of Works_On : [(2895000, )]
size of Dept_Locations : [(580012, )]


In [31]:

data = cursor.execute('select count(*) from Project where Pnumber < 50')
print(list(data))


[(49, )]
